# Double Well Potential with MCMC

Consider the double well potential,

$$
V(x) = ax^4 - 2x^2 + \frac{1}{a}
$$

where $x$ is the position of the particle, and we set $m = \hbar = 1$ and $a = 0.4$. See Refs. [1, 2] for discussions of similar problems.

Use the path integral formulation with imaginary time $\tau$,

$$
Z = \int Dx(\tau) \exp \left[ -\frac{1}{\hbar} \int_0^{t_b} L_E(x(\tau))d\tau \right]
$$

$$
= \lim_{\delta\tau \to 0} \int \cdots \int dx_0 \cdots dx_{N-1} \left( \frac{2\pi\hbar\delta\tau}{m} \right)^{-\frac{N}{2}} \exp \left[ -\frac{1}{\hbar} \sum_{i=1}^{N} \frac{m}{2\delta\tau} (x_i - x_{i-1})^2 + \delta\tau V\left(\frac{x_{i-1} + x_i}{2}\right) \right]
$$

where $L_E(x(\tau)) = \frac{m}{2} \left( \frac{dx}{d\tau} \right)^2 + V(x(\tau))$ is the Euclidean Lagrangian, imaginary time is discretized with $N$ increments, $\tau_a = 0$, and $\tau_b = N\delta\tau = \hbar\beta$. So the probability of a given path $(x_0, x_1, \ldots, x_{N-1})$ is

$$
p(x_0, \ldots, x_{N-1}) \propto \exp \left[ -\frac{1}{\hbar} \sum_{i=1}^{N} \frac{m}{2\eta} (x_i - x_{i-1})^2 + \eta V\left(\frac{x_{i-1} + x_i}{2}\right) \right]
$$


## Problem A

Evaluate the ground state energy and probability distribution of the particle using Markov chain Monte Carlo with the Metropolis-Hastings algorithm in the large-$\tau_b$ (imaginary time) limit. Describe your strategy for determining $\tau$, the initial conﬁguration, burn-in steps, hit size, number of sweeps, and thinning (if any).

## Problem B

Plot the ground state probability distribution and compare it with the expected form.

## Problem C

Calculate the energy and probability distribution of the particle from the same simulation code for a smaller value of $\tau_b$. What is the expected probability distribution in this case? What does this correspond to in terms of a statistical mechanics interpretation?

## Bibliography

[1] S. Mittal, M. J. Westbroek, P. R. King, and D. D. Vvedensky, “Path integral Monte Carlo method for the quantum anharmonic oscillator”, [Eur. J. Phys. 41, 055401 (2020)](https://doi.org/10.1088/1361-6404/ab9a66).


[2] R. Rodgers and L. Raes, *Monte Carlo simulations of harmonic and anharmonic oscillators in discrete Euclidean time*, DESY Summer Student Programme (2014).